# pyGemPick Tutorial 4: Calculating Cross Correlation & K-function using detected gold particle center coordinates

From Tutorial 3 we sucessfully recorded the spatial point coordinates of detected particles in our test data set. To determine their pair wise correlation statistics, a normalized version of an edited K statistic by philmonenko et al in 2000 was used. This was embedded in the csv2pcf function which takes data from csv produced by bin2csv() and outputs non-normalized k and pcf (pair-correlation) spatially invairenttime series for set of images...
Analyzed by bin2csv. Example output provided in docs. 

* dr (r) is the donut width as defined by [philmonenko et al, 2000](http://nucleus.img.cas.cz/pdf_publications/PHI_Statistical%20evaluation%20of%20Colocalization%20Patterns_01.pdf)
* a is the hight of the image in pixels 
* b is the width of the image in pixels
* N is the number of images processed in the dataset 
* ni is the total number of particles detected in the dataset 
*  \lambda is the density of particles on an image in the dataset.

$$\begin{align} \lambda = \frac{ni}{N} \end{align}$$

* \gamma is the window covariogram correcting for boundary conditions.

$$\begin{align}\gamma = A - \frac{2}{\pi*({a}+{b})*{r}} + \frac{r^2}{\pi}\end{align}$$

* pcf is the normalized pairwise correlation statistic

$$\begin{align}{K}_j^{-1}(dr) = \frac{1}{2\pi*{N}*\lambda^2}\sum_{i=1}^{N}\sum_{x \neq  y} \frac{1\,if\,r' \leq d(x,y) < r"}{d(x,y) * y(d(x,y))}\end{align}$$

### The csv2pcf() function is defined below:

```python

import pygempick.spatialstats as spa

def csv2pcf(data, dr, max_radius):
    
    Takes data from csv produced by bin2csv() and outputs non-normalized k 
    and pcf (pair-correlation) spatially invairenttime series for set of images...
    Analyzed by bin2csv. Example output provided in docs.
    
    dr is the donut width as defined by philmonenko et al, 2000
    
    
    N = int(input('How Many Processed Images in this set?'))
    ni = int(input('How many particles were detected?')) #nuber of particles counted in test set

    
    data1 = pd.read_csv(data, header=None, skiprows=1)
    data = pd.DataFrame(data1)
    
    a = 776 ## number of y pixels in image (height)
    b = 1018 ##number of x pixels in image (width)
    
    #explain why this function is required for statistical detection of immunogold clusters in the data...
    
    l = (1/(a*b*N))*ni #average density of particles lables across this test set of images (can calculate with or without area)
    dni = 50*ni/1000. #false negatives missed by picker per 1000
    
    #correct the boundary effect of image by using the geometric covariogram of the
    # window - defined in Ripley 1988 -- this is given by the gamma function in A_correlation_test
    
    k = []
    dk = []
    pcf = []
    dpcf = []
    
    for r in range(0, max_radius ,dr): ##analyze the clustering in a given region between two circles. 
        
        kc = 0
        ki = 0
        
        for p in range(0,len(data),2):
            
            #x,y coordinates of detected immunogold keypoints occur in set of twos in the loaded csv file
            x = np.array(data[p][~np.isnan(data[p])])
            y = np.array(data[p+1][~np.isnan(data[p+1])])
            
            if len(x) > 0:
                
                for i in range(len(x)-1): ##ensure that there are no duplicates while comparing all keypoints
                
                    for j in range(i+1, len(x)):
                            
                        rad = np.sqrt((x[i]-x[j])**2 + (y[i] - y[j])**2)
                            
                        if r < rad <= r+dr: #if radius is less than this area b/w circles
                                
                            kc+= 1/gamma(a,b,rad) #classical K function
                            ki+= 1/(rad*gamma(a,b, rad)) #pair correlation function
                        
                        else:
                            kc+=0   
                            ki+=0
                            
                print(p)
                
        a = kc*(1/(N*l))
        da = a*(dni/ni)
        b = ki*(1/(N*l*2*np.pi))
        db = b*(dni/ni)
       
        k.append(a) #classical K function
        dk.append(da)
        pcf.append(b)
        dpcf.append(db)
        
        print(k,r) 
    
    return k, pcf , dk, dpcf
```

In [1]:
#import required modules
import glob
import pandas as pd
import numpy as np
import pygempick.spatialstats as spa
import pygempick.modeling as mod
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
#create empty pandas Dataframe
tosave = pd.DataFrame()

In [3]:
# defines the binwidth you will use in the calculation of Ripley's spatial statistics!

dr = 5
maxr = 200

k, pcf, dk, dpcf = spa.csv2pcf('/home/joseph/Documents/pygempick/supdocs/v30m_anti_july418_31_79_60_60_1528.csv', dr, maxr)

How Many Processed Images in this set?175
How many particles were detected?155
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
[5.309648986626465e-07] 0
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
[5.309648986626465e-07, 0.00047308083169960623] 5
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
[5.309648986626465e-07, 0.00047308083169960623, 0.0022234873230338207] 10
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
[5.309648986626465e-07, 0.00047308083169960623, 0.0022234873230338207, 0.0020920989895361094] 15
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
[5.309648986626465e-07, 0.00047308083169960623, 0.0022234873230338207, 0.0020920989895361094, 0.0017532565781075598] 20
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
[5.309648986626465e-07, 0.00047308083169960623, 0.0022234873230338207, 0.0020920989895361094, 0.0017532565781075598, 0.0016053064476339919] 25
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
[5.309648986626465e-07, 0.

24
26
28
30
32
34
36
38
[5.309648986626465e-07, 0.00047308083169960623, 0.0022234873230338207, 0.0020920989895361094, 0.0017532565781075598, 0.0016053064476339919, 0.0009109156230501872, 0.0006063371628264813, 0.00044158505316739153, 0.0003791741588254205, 0.0003183854889832933, 0.00019491488321718965, 0.00010869856870179323, 8.499876803938106e-05, 6.123589204962426e-05, 8.412506163803835e-05, 2.645870817974947e-05, 1.3434970271318338e-05, 2.87834430138586e-05, 2.7354571084435112e-05, 1.8584359569980804e-06, 1.0673222324903368e-05, 1.4521536241723726e-05, 9.999697217047104e-06, 1.6048474481616384e-05, 1.7441720245368908e-05, 1.2532436498395193e-05, 1.7100916240617518e-05, 1.304399671596162e-05, 7.42231625717325e-06, 7.737371763920157e-06, 1.4832557837394165e-05, 1.1503751907627501e-05, 1.0891178580726515e-05] 165
0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
[5.309648986626465e-07, 0.00047308083169960623, 0.0022234873230338207, 0.0020920989895361094, 0.0017532565781075598, 0.0

In [4]:
# when datasets have large amounts of partcles pair correlation can be easily calculated!
# As seen here - datasets with limited amounts of data have little to no valid results!

In [5]:
name = input('Test set name:')  

Test set name:ANTITTR


In [6]:
#calculate error in k function as the standard deviation of the completed calculation
stdk = np.sqrt(sum(k/sum(k))/len(k)) 


In [7]:
#calculate error in pair correlation function function as the standard deviation of the completed calculation
stdpcf = np.sqrt(sum(pcf/sum(pcf))/len(pcf))


In [8]:
npcf = np.array(pcf)/sum(pcf) #normalized pcf values

In [9]:
sumd = np.sqrt(sum(np.array(dpcf)**2)) #propogatin the normalized pcf error 
dnpcf = npcf*np.sqrt((np.array(dpcf)/pcf)**2 + (sumd/sum(pcf))**2) 

In [10]:
df = pd.DataFrame({'dr-{}'.format(name): np.linspace(1,101, len(pcf)), 
                   'PCF-{}'.format(name): npcf, 'dpcf-{}'.format(name): dnpcf})

In [11]:
tosave = pd.concat([tosave,df], ignore_index=False, axis=1)

In [12]:
print(npcf,dnpcf)

[9.10503446e-03 1.22976901e-01 2.94183748e-01 2.07234679e-01
 1.29928796e-01 1.00612899e-01 4.80712224e-02 2.76160814e-02
 1.75685459e-02 1.34764724e-02 1.03643784e-02 5.75323127e-03
 2.95834095e-03 2.14987479e-03 1.43282630e-03 1.84603209e-03
 5.42311608e-04 2.63387145e-04 5.32965326e-04 4.74456539e-04
 3.02790256e-05 1.70175202e-04 2.20499701e-04 1.42850862e-04
 2.21878065e-04 2.32721703e-04 1.59887098e-04 2.11333748e-04
 1.56180132e-04 8.54571364e-05 8.57518140e-05 1.60934752e-04
 1.20415680e-04 1.10927796e-04 1.49752621e-04 1.15910963e-04
 1.18236661e-04 1.82339775e-04 1.15296342e-04 1.16987249e-04] [4.93570434e-04 6.66639570e-03 1.59472653e-02 1.12338850e-02
 7.04324764e-03 5.45407626e-03 2.60586978e-03 1.49702687e-03
 9.52364858e-04 7.30539612e-04 5.61837602e-04 3.11874147e-04
 1.60367282e-04 1.16541529e-04 7.76713918e-05 1.00070666e-04
 2.93979092e-05 1.42778271e-05 2.88912611e-05 2.57195863e-05
 1.64138114e-06 9.22494571e-06 1.19529622e-05 7.74373367e-06
 1.20276813e-05 1.26154

In [13]:
tosave.to_csv('/home/joseph/Desktop/pcf-dr{}-error-single.csv'.format(dr),index=False)

In [14]:
#Have to look at this to calculate correctly when I recieve newer data!